# D’oh! Unevenly spaced time series analysis of The Simpsons in Pandas
#### [PyData Seattle 2017 tutorial](https://pydata.org/seattle2017/schedule/presentation/104/)
#### [Joe McCarthy](http://interrelativity.com/joe/), Data Scientist, [Indeed](https://www.indeed.com)



In [ ]:
import datadotworld as dw

In [ ]:
lds = dw.load_dataset('data-society/the-simpsons-by-the-data') #, force_update=True)

In [ ]:
! ls -l ~/.dw/cache/data-society/the-simpsons-by-the-data/latest/data

In [ ]:
df = lds.dataframes['simpsons_script_lines']

TODO: Investigate warnings:

```/Users/joem/anaconda/lib/python2.7/site-packages/datadotworld/models/dataset.py:192: UserWarning: Unable to set data frame dtypes automatically using simpsons_script_lines schema. Data types may need to be adjusted manually. Error: Integer column has NA values in column 7
  'Error: {}'.format(resource_name, e))
/Users/joem/anaconda/lib/python2.7/site-packages/datadotworld/util.py:136: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return self._loader_func()```

In [ ]:
df.info()

9 columns have type `object`, which is usually an indicator that the column contains a mix of data types. 

Sometimes this happens when columns include null (`NaN`) values, so let's show the number of null values along with the number of data types, and include the number of unique values in each column. 

TODO: Is there a better way to do this?

In [ ]:
for i, column_label in enumerate(df.columns):
    print('{:2d}: {:20} {:6d} unique values, {:6d} null values, {:2d} data type(s)'.format(
        i,
        column_label + ':', 
        df[column_label].nunique(),
        df[column_label].isnull().sum(),
        len(df[column_label].apply(lambda x: type(x)).value_counts())))

Let's see what kinds of values are stored in the columns that have mixed data types.

In [ ]:
for i, column_label in enumerate(df.columns):
    if len(df[column_label].apply(lambda x: type(x)).value_counts()) > 1:
        print(i, column_label)
        print(df[column_label].apply(lambda x: type(x)).value_counts())
        print()

## Data Quality Analysis

There are a number of columns which have more than one data type, e.g., `str` and `float`. Before diving into the time series analysis, we'll spend a little time doing some data quality analysis.

In [ ]:
df.head()

### `timestamp_in_ms`

In [ ]:
df.timestamp_in_ms.apply(lambda x: type(x)).value_counts()

In [ ]:
df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str))].timestamp_in_ms.nunique()

In [ ]:
df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str))].timestamp_in_ms.head()

Looks like some of the `str` values are `'0'`. Are they all `'0'`?

In [ ]:
(len(df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and x.strip() == '0')]),
 len(df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and x.strip() != '0')]))

What are some of the other `str` values?

In [ ]:
df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and x.strip() != '0')].timestamp_in_ms.head()

Can we just convert them all to integers?

In [ ]:
df.loc[:, 'timestamp_in_ms'] = df.timestamp_in_ms.astype('int64')

Apparently not. `'Springfield Elementary School'` is a value that cannot be converted to an integer. How many non-numeric values are there?

In [ ]:
len(df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and not x.isdigit())])

Only one. What does that row look like?

In [ ]:
df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and not x.isdigit())]

In [ ]:
index_position = df[df.timestamp_in_ms.apply(lambda x: isinstance(x, str) and not x.isdigit())].index.values[0]
index_position

In [ ]:
df.loc[index_position-2:index_position+2, :]

One way to fix this is to simply delete the row

In [ ]:
# df = df[df.timestamp_in_ms.apply(lambda x: not isinstance(x, str) or x.isdigit())]

Another way to fix this is to figure out which column values are wrong and correct them.

In [ ]:
df.columns

We can use the [`pandas.Index.get_loc()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Index.get_loc.html) method to find the position of `raw_location_text` the DataFrame columns Index.

In [ ]:
df.columns.get_loc('raw_location_text'), len(df.columns)

In [ ]:
# shift some column values over by 5
for i in range(df.columns.get_loc('raw_location_text'), len(df.columns)):
    df.iloc[index_position, i] = df.iloc[index_position, i - 5]

# compute an intermediate timestamp_in_ms value based on preceding and succeeding timestamp_in_ms values
next_timestamp = int(df.loc[index_position + 1, 'timestamp_in_ms'])
prev_timestamp = int(df.loc[index_position - 1, 'timestamp_in_ms'])
timestamp_delta = next_timestamp - prev_timestamp
timestamp_increment = int(round(timestamp_delta / 2000, 0) * 1000)  # timestamps all seem to be in seconds
new_timestamp = prev_timestamp + timestamp_increment 
timestamp_delta, timestamp_increment, prev_timestamp, new_timestamp, next_timestamp
df.loc[index_position, 'timestamp_in_ms'] = str(new_timestamp)

# use values in preceding row for other columns
for i in range(df.columns.get_loc('speaking_line'), df.columns.get_loc('raw_character_text')):
    df.iloc[index_position, i] = df.iloc[index_position - 1, i]

In [ ]:
df.loc[index_position-2:index_position+2, :]

Now we can convert the `timestamp_in_ms` values into integers (without exceptions being raised).

In [ ]:
df.loc[:, 'timestamp_in_ms'] = df.timestamp_in_ms.astype('int64')

### `speaking_line`

In [ ]:
df.speaking_line.nunique()

The `speaking_line` column contains 4 different values, even though the `bool` data type only has two.

In [ ]:
df.speaking_line.value_counts()

The `speaking_line` column has both Pythonistic representations of `bool` values (`True` and `False`) and lowercase `str` values more like the values used in other programming languages, e.g., Java (`true` and `false`).

In [ ]:
df[df.speaking_line.apply(lambda x: isinstance(x, bool))].speaking_line.value_counts()

In [ ]:
df[df.speaking_line.apply(lambda x: isinstance(x, str))].speaking_line.value_counts()

First, we can identify rows containig either `True` or `true` values

In [ ]:
df[df.speaking_line.apply(
    lambda x: isinstance(x, bool) and x is True
        or isinstance(x, str) and x == 'true')].speaking_line.value_counts()

In [ ]:
df.loc[:, 'speaking_line'] = df.speaking_line.apply(
    lambda x: isinstance(x, bool) and x is True 
        or isinstance(x, str) and x == 'true')

Verify that we have only `bool` values in the column now.

In [ ]:
df.speaking_line.value_counts()

### `character_id`

In [ ]:
df.character_id.apply(lambda x: type(x)).value_counts()

In [ ]:
df.character_id.nunique()

In [ ]:
df[df.character_id.apply(lambda x: isinstance(x, str))].character_id.nunique()

In [ ]:
len(df[df.character_id.apply(lambda x: isinstance(x, str) and not x.isdigit())])

All the `str` values are numeric, so perhaps we can convert everything to integer.

In [ ]:
df.loc[:, 'character_id'] = df.character_id.astype('int64')

We cannot convert null (`NaN`) values to integer.

However, it appears that `0` is not used for any valid `character_id` value, so we can use this for the null values.

In [ ]:
df[df.character_id == 0].raw_character_text

In [ ]:
df.loc[:, 'character_id'] = df.character_id.fillna(0).astype('int64')

Verify that we have only a single data type now.

In [ ]:
df.character_id.apply(lambda x: type(x)).value_counts()

### `location_id`

The `location_id` field has floats.

In [ ]:
df.location_id.apply(lambda x: type(x)).value_counts()

As with the `character_id` column,it appears that 0 is not used for any id value, so we can use this for the null values.

In [ ]:
df[df.location_id == 0].raw_location_text

In [ ]:
df.loc[:, 'location_id'] = df.location_id.fillna(0).astype('int64')

### `raw_character_text`

In [ ]:
df.raw_character_text.apply(lambda x: type(x)).value_counts()

In [ ]:
len(df[df.raw_character_text.isnull()])

It appears that all the `float` values are null (`NaN`), so we can convert them to empty strings.

In [ ]:
df.raw_character_text.fillna('', inplace=True)

Verify that we have only a single data type now.

In [ ]:
df.raw_character_text.apply(lambda x: type(x)).value_counts()

### `raw_location_text`

In [ ]:
df.raw_location_text.apply(lambda x: type(x)).value_counts()

In [ ]:
len(df[df.raw_location_text.isnull()])

In [ ]:
df.raw_location_text.fillna('', inplace=True)

In [ ]:
df.raw_location_text.apply(lambda x: type(x)).value_counts()

### `spoken_words`

In [ ]:
df.spoken_words.apply(lambda x: type(x)).value_counts()

In [ ]:
len(df[df.spoken_words.isnull()])

In [ ]:
df.spoken_words.fillna('', inplace=True)

In [ ]:
df.spoken_words.apply(lambda x: type(x)).value_counts()

### `normalized_text`

In [ ]:
df.normalized_text.apply(lambda x: type(x)).value_counts()

In [ ]:
len(df[df.normalized_text.isnull()])

In [ ]:
df.normalized_text.fillna('', inplace=True)

In [ ]:
df.normalized_text.apply(lambda x: type(x)).value_counts()

In [ ]:
len(df[(df.normalized_text == '') & (df.spoken_words != '')])

The number of null values in `normalized_text` (26183) was higher than the number of null values in `spoken_words`. What do the non-null `spoken_words` values look like for the rows in which the `normalized_text` values are null?

In [ ]:
df[(df.normalized_text == '') & (df.spoken_words != '')].spoken_words

Looks like puncutation that was not removed during normalization, so let's convert all of these to empty strings.

In [ ]:
df.loc[(df.normalized_text == '') & (df.spoken_words != ''), 'spoken_words'] = ''

In [ ]:
len(df[(df.normalized_text == '') & (df.spoken_words != '')])

### `word_count`

In [ ]:
df.word_count.apply(lambda x: type(x)).value_counts()

In [ ]:
import numpy as np

In [ ]:
df[df.word_count.apply(lambda x: isinstance(x, np.float64))]

In [ ]:
df[df.word_count.apply(lambda x: isinstance(x, np.float64))].word_count

In [ ]:
len(df[df.word_count.isnull()])

In [ ]:
df.loc[:, 'word_count'] = df.word_count.fillna(0).astype('int64')

In [ ]:
len(df[df.word_count.apply(lambda x: isinstance(x, str) and not x.isdigit())])

In [ ]:
df[df.word_count.apply(lambda x: isinstance(x, str) and not x.isdigit())].word_count

In [ ]:
df[df.word_count.apply(lambda x: isinstance(x, str) and not x.isdigit())]

Looks like this is probably an underlying problem with the CSV file.

In [ ]:
! grep "FIRST ANNUAL" ~/.dw/cache/data-society/the-simpsons-by-the-data/latest/data/simpsons_script_lines.csv

In this case, rather than repairing the CSV file, we'll just drop the rows that appear to have invalid values for `word_count`. I always check for the size of the DataFrame that results from dropping the rows before assigning the resulting DataFrame to the variable.

In [ ]:
len(df[df.word_count.apply(lambda x: not isinstance(x, str) or x.isdigit())])

In [ ]:
df = df[df.word_count.apply(lambda x: not isinstance(x, str) or x.isdigit())]

Now we can convert everything to integer values

In [ ]:
df.loc[:, 'word_count'] = df.word_count.fillna(0).astype('int64')

Are there any outlying values, e.g., word_count values that are 1000+?

In [ ]:
len(df[df.word_count>=1000])

In [ ]:
df[df.word_count>=1000]

In [ ]:
df = df[df.word_count<1000]

### Sorting the script lines

The DataFrame rows are not in sorted order. 

In [ ]:
df.head()

In [ ]:
df[(df.episode_id==1) & (df.number<5)]

The `id` field represents a chronological ordering of the script lines, so we will sort the DataFrame by the values in this field to simplify later processing

In [ ]:
df = df.sort_values('id').reset_index(drop=True)

In [ ]:
df.head()